In [143]:
!pip install pandas pytorch-frame catboost scikit-learn


[notice] A new release of pip is available: 24.1.2 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [1]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
import catboost
from torch_frame.utils import infer_df_stype

In [2]:
pd.set_option('display.max_columns', 1000)

In [3]:
df = pd.read_parquet('https://kurve-customers.s3.amazonaws.com/4e1a245a-3065-4600-bb0e-a92e06ee835c/1/output/user_engage_odsc_eval')
test = pd.read_parquet('https://kurve-customers.s3.amazonaws.com/4e1a245a-3065-4600-bb0e-a92e06ee835c/1/output/user_engage_odsc_test')

In [4]:
print(test.shape)
print(df.shape)

(87789, 229)
(67191, 232)


In [5]:
[c for c in test.columns if 'had_engagement' in c]

['User_had_engagement']

In [6]:
print(test['User_had_engagement'].sum()/len(test))
print(df['User_had_engagement'].sum()/len(test))

0.027338277005091755
0.0295822939092597


In [7]:
stypes = infer_df_stype(test)

In [8]:
features = [
    k for k,v in stypes.items()
    if str(v) == 'numerical'
    and k not in ['User_Id', 'User_AccountId']
    and 'label' not in k and k != 'had_engagement']

In [9]:
len(features)

190

In [10]:
features = [c for c in features if c in df.columns]

In [11]:
target = 'User_had_engagement'


import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool

# -------------------------------------------------
# 1. Split off the *final* test set (once!)
# -------------------------------------------------
X_train_full, X_test, y_train_full, y_test = train_test_split(
    df[features], df[target],
    test_size=0.20,          # 20 % held-out test
    stratify=df[target],
    random_state=42
)

# -------------------------------------------------
# 2. K-Fold CV on the remaining 80 %
# -------------------------------------------------
k = 3                                   # change to 10, etc.
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)

fold_aucs = []
test_preds = np.zeros(len(X_test))      # out-of-fold predictions on *test*
oof_preds = np.zeros(len(X_train_full)) # optional: OOF on training data

# CatBoost pools (optional but faster)
train_pool = Pool(X_train_full, y_train_full)  # , cat_features=cat_features)

for fold, (idx_tr, idx_va) in enumerate(skf.split(X_train_full, y_train_full), 1):
    print(f"\n=== Fold {fold} ===")
    
    X_tr, X_va = X_train_full.iloc[idx_tr], X_train_full.iloc[idx_va]
    y_tr, y_va = y_train_full.iloc[idx_tr], y_train_full.iloc[idx_va]

    # -----------------------------------------------------------------
    # 3. Fit on the *training* split of this fold
    # -----------------------------------------------------------------
    mdl = catboost.CatBoostClassifier(
        # objective & metrics
        loss_function="Logloss",
        eval_metric="AUC",
        custom_metric=["AUC", "PRAUC", "F1", "Recall", "Precision", "Logloss"],
        use_best_model=True,
    
        # capacity vs regularization
        iterations=2000,
        learning_rate=0.05,
        depth=5,
        l2_leaf_reg=10,
    
        # randomness / bagging / feature subsampling
        bootstrap_type="Bayesian",
        bagging_temperature=0.75,
        rsm=0.8,
        random_strength=0.5,
    
        # class imbalance handling
        auto_class_weights="Balanced",
        #class_weights=[1,36],
    
        # borders / leaves
        feature_border_type="GreedyLogSum",
        min_data_in_leaf=20,
        boosting_type="Plain",
    
        # early stopping
        od_type="Iter",
        od_wait=150,
    
        verbose=200
    )
    mdl.fit(
        X_tr, y_tr,
        eval_set=(X_va, y_va),
        use_best_model=False,
        verbose=False
    )

    # -------------------------------------------------
    # 4. Validation AUC for this fold
    # -------------------------------------------------
    val_pred = mdl.predict_proba(X_va)[:, 1]
    val_auc  = roc_auc_score(y_va, val_pred)
    fold_aucs.append(val_auc)
    print(f"Fold {fold} validation AUC : {val_auc:.4f}")

    # -------------------------------------------------
    # 5. Accumulate predictions on the *final* test set
    # -------------------------------------------------
    test_preds += mdl.predict_proba(X_test)[:, 1] / k

    # (optional) OOF on training data
    oof_preds[idx_va] = val_pred

# -------------------------------------------------
# 6. Final metrics
# -------------------------------------------------
print("\n=== CV Summary ===")
print(f"Mean CV AUC : {np.mean(fold_aucs):.4f} ± {np.std(fold_aucs):.4f}")
print(f"Folds AUC   : {[f'{a:.4f}' for a in fold_aucs]}")

test_auc = roc_auc_score(y_test, test_preds)
print(f"\nFinal test AUC (averaged over {k} folds): {test_auc:.4f}")

# -------------------------------------------------
# 7. (Optional) Refit on the *whole* train_full for deployment
# -------------------------------------------------
final_mdl = catboost.CatBoostClassifier(
        # objective & metrics
        loss_function="Logloss",
        eval_metric="AUC",
        custom_metric=["AUC", "PRAUC", "F1", "Recall", "Precision", "Logloss"],
        #use_best_model=True,
    
        # capacity vs regularization
        #iterations=2000,
        iterations=int(mdl.best_iteration_ * 1.1),  # a bit more than best
        learning_rate=0.05,
        depth=5,
        l2_leaf_reg=10,
    
        # randomness / bagging / feature subsampling
        bootstrap_type="Bayesian",
        bagging_temperature=0.75,
        rsm=0.8,
        random_strength=0.5,
    
        # class imbalance handling
        auto_class_weights="Balanced",
        #class_weights=[1,36],
    
        # borders / leaves
        feature_border_type="GreedyLogSum",
        min_data_in_leaf=20,
        boosting_type="Plain",
    
        # early stopping
        od_type="Iter",
        od_wait=150,
    
        verbose=200
    )
final_mdl.fit(X_train_full, y_train_full)


=== Fold 1 ===
Fold 1 validation AUC : 0.8826

=== Fold 2 ===
Fold 2 validation AUC : 0.8825

=== Fold 3 ===
Fold 3 validation AUC : 0.8764

=== CV Summary ===
Mean CV AUC : 0.8805 ± 0.0029
Folds AUC   : ['0.8826', '0.8825', '0.8764']

Final test AUC (averaged over 3 folds): 0.8897
0:	total: 8.42ms	remaining: 1.18s
140:	total: 1.72s	remaining: 0us


In [12]:
test['pred'] = final_mdl.predict_proba(test[features])[:,1]


In [13]:
from sklearn import metrics# import roc_auc_score

In [14]:
print(metrics.roc_auc_score(test[target], test['pred']))

0.8981103489130138


In [15]:
list(reversed(sorted(zip(final_mdl.feature_names_, final_mdl.feature_importances_), key=lambda x: x[1])))

[('ph_seconds_since_last', 5.496552222405458),
 ('post_seconds_since_last', 5.237788683381555),
 ('Badg_badge_incidence_sum', 4.344623738436722),
 ('Badg_Name_count', 4.171805141829088),
 ('Badg_Class_sum', 4.162442162738467),
 ('post_age_days_min', 4.11228759746425),
 ('comm_num_events_730d', 2.9207586203767804),
 ('comm_seconds_since_last', 2.4787583942651326),
 ('ph_365dv730_change', 2.477859931940493),
 ('Badg_badge_incidence_min', 2.4461949664558804),
 ('comm_age_days_min', 2.4405801352580534),
 ('post_num_events_365d', 2.4152110720200475),
 ('post_num_events_730d', 2.3759698740148156),
 ('comm_num_events_180d', 2.0615912577255084),
 ('ph_Id_count', 2.041156218755769),
 ('ph_num_events_365d', 1.9119812633476319),
 ('post_Id_count', 1.794460053173382),
 ('Badg_Id_count', 1.7756449012301734),
 ('subvote_seconds_since_last_min', 1.6831721105003858),
 ('badg_age_days_max', 1.5747014394711936),
 ('comm_num_events_365d', 1.5239645384355158),
 ('badg_age_days_min', 1.4745800216450424),
 